In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t

# a. Assume
i. Initial stock price is $100
ii. Strike price of the option is $100
iii. Risk free rate is 0.0525%
iv. The stock does not pay dividends.
v. The expected return on the stock is 10%.
vi. Implied volatility of the option equals the realized volatility of the stock which is
25% annually.
vii. The option expires in 1 year.
viii. You will hold the portfolio until option expiration.
ix. You are allowed to short either the option or the stock, but no weight should be <-1

In [11]:
# 参数定义
initial_stock_price = 100  # 初始股价
strike_price = 100         # 期权执行价
risk_free_rate = 0.0525 / 100  # 无风险利率（年化）
expected_return_stock = 0.10   # 股票期望回报率（年化）
volatility_stock = 0.25        # 股票的年化波动率
time_to_expiry = 1            # 期权到期时间（1年）
n_simulations = 100000        # 模拟次数
alpha = 0.01                  # ES置信水平

# 期权定价模型（Black-Scholes公式）
def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# 计算期望短缺 (Expected Shortfall, ES)
def calculate_es(returns, alpha=0.01):
    """
    根据模拟的收益数据计算期望短缺 (ES)。
    参数:
        returns (numpy array): 模拟的收益数组。
        alpha (float): VaR 的置信水平，默认为 0.01 (1%)。
    返回:
        es (float): 期望短缺值 (ES)。
    """
    # 计算 VaR (分位数)
    var_threshold = np.percentile(returns, alpha * 100)
    # 筛选低于 VaR 的收益值
    tail_losses = returns[returns <= var_threshold]
    # 计算 ES
    es = np.mean(tail_losses) if len(tail_losses) > 0 else var_threshold
    return es

# 计算期权初始价格
initial_put_price = black_scholes_put(
    S=initial_stock_price, 
    K=strike_price, 
    T=time_to_expiry, 
    r=risk_free_rate, 
    sigma=volatility_stock
)

print(f"初始股价: ${initial_stock_price}")
print(f"初始看跌期权价格: ${initial_put_price:.2f}")


初始股价: $100
初始看跌期权价格: $9.92


# b. Simulate 100,000 returns of both the stock and option through the holding period. Calculate the covariance between these

In [12]:
# 模拟股票价格路径
np.random.seed(42)  # 固定随机种子
z_stock = np.random.normal(size=n_simulations)
stock_returns = expected_return_stock * time_to_expiry + \
                volatility_stock * np.sqrt(time_to_expiry) * z_stock
simulated_stock_prices = initial_stock_price * np.exp(stock_returns)

# 模拟期权到期价格
def put_payoff(S, K):
    return np.maximum(K - S, 0)

simulated_put_prices = put_payoff(simulated_stock_prices, strike_price)

# 计算持有期收益率
stock_returns_held = (simulated_stock_prices - initial_stock_price) / initial_stock_price
put_returns_held = (simulated_put_prices - initial_put_price) / initial_put_price

# 计算协方差
covariance = np.cov(stock_returns_held, put_returns_held)[0, 1]
print(f"股票和期权的收益协方差: {covariance:.4f}")


股票和期权的收益协方差: -0.1857


# c. Calculate the maximum Sharpe Ratio Portfolio
i. (HINT: Because you have 2 Assets, you can quickly sweep the potential weights to find maximum values instead of using an optimizer. Weight_Stock = 1- Weight_Option)

In [13]:
# 定义权重范围
weights_stock = np.linspace(-1, 1, 500)  # 股票权重
weights_option = 1 - weights_stock       # 期权权重

# 扩展权重数组的形状，使其与收益数组形状一致
weights_stock = weights_stock.reshape(-1, 1)  # 变为 (500, 1)
weights_option = weights_option.reshape(-1, 1)  # 变为 (500, 1)

# 计算投资组合收益
portfolio_returns = weights_stock * stock_returns_held + weights_option * put_returns_held  # 形状为 (500, 100000)

# 计算投资组合的平均收益和波动率
portfolio_mean = np.mean(portfolio_returns, axis=1)  # 对每个权重组合计算平均收益
portfolio_volatility = np.std(portfolio_returns, axis=1)  # 对每个权重组合计算波动率

# 计算 Sharpe 比率
sharpe_ratios = (portfolio_mean - risk_free_rate) / portfolio_volatility

# 找到最大 Sharpe 比率的权重
max_sharpe_idx = np.argmax(sharpe_ratios)
max_sharpe_weights = (weights_stock[max_sharpe_idx][0], weights_option[max_sharpe_idx][0])

print(f"最大 Sharpe 比率组合: 股票权重 = {max_sharpe_weights[0]:.4f}, 期权权重 = {max_sharpe_weights[1]:.4f}")
print(f"最大 Sharpe 比率: {sharpe_ratios[max_sharpe_idx]:.4f}")


最大 Sharpe 比率组合: 股票权重 = 1.0000, 期权权重 = 0.0000
最大 Sharpe 比率: 0.4833


# d. Change the Sharpe Ratio formula to use the ES (alpha=0.01) from your simulated values instead of the portfolio volatility. What weights maximize this reward/risk ratio?

In [14]:
# 定义 ES 奖励/风险比率
es_ratios = []

for w_stock, w_option in zip(weights_stock.flatten(), weights_option.flatten()):  # 展平权重数组
    portfolio_returns = w_stock * stock_returns_held + w_option * put_returns_held
    es = calculate_es(portfolio_returns, alpha)
    reward = np.mean(portfolio_returns)
    es_ratios.append(reward / -es)  # 奖励/风险比率

# 找到最大 ES 奖励/风险比率的权重
max_es_idx = np.argmax(es_ratios)
max_es_weights = (weights_stock[max_es_idx][0], weights_option[max_es_idx][0])

print(f"最大 ES 奖励/风险比率组合: 股票权重 = {max_es_weights[0]:.4f}, 期权权重 = {max_es_weights[1]:.4f}")
print(f"最大 ES 奖励/风险比率: {es_ratios[max_es_idx]:.4f}")


最大 ES 奖励/风险比率组合: 股票权重 = 0.9078, 期权权重 = 0.0922
最大 ES 奖励/风险比率: 0.8951


# e. Why are these values so different?

In [15]:
# 打印两种方法的最佳权重结果
print(f"最大 Sharpe 比率组合权重: 股票 {max_sharpe_weights[0]:.4f}, 期权 {max_sharpe_weights[1]:.4f}")
print(f"最大 ES 奖励/风险比率组合权重: 股票 {max_es_weights[0]:.4f}, 期权 {max_es_weights[1]:.4f}")

# 差异分析
print("两种方法的差异是由于 Sharpe 比率使用波动率作为风险指标，而 ES 奖励/风险比率更关注尾部风险，")
print("因此在极端市场情况下，ES 方法可能更适合评估风险。")


最大 Sharpe 比率组合权重: 股票 1.0000, 期权 0.0000
最大 ES 奖励/风险比率组合权重: 股票 0.9078, 期权 0.0922
两种方法的差异是由于 Sharpe 比率使用波动率作为风险指标，而 ES 奖励/风险比率更关注尾部风险，
因此在极端市场情况下，ES 方法可能更适合评估风险。
